In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_circles
from imblearn.under_sampling import (NearMiss, InstanceHardnessThreshold, EditedNearestNeighbours, 
                                     TomekLinks, AllKNN, ClusterCentroids)
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier

# 円形の分布を生成する関数（オフセット付き）
def make_circular_data(n_samples, noise, random_state, x_offset=0, y_offset=0):
    X, y = make_circles(n_samples=n_samples, noise=noise, factor=0.5, random_state=random_state)
    X[:, 0] += x_offset
    X[:, 1] += y_offset
    return X, y

# 設定値
n_major = 2000
n_minor = 400
target_ratio = 0.2

# majorクラスとminorクラスのデータを生成（オフセットを適用）
X_major, _ = make_circular_data(n_samples=n_major, noise=0.1, random_state=42, x_offset=-0.3)
X_minor, _ = make_circular_data(n_samples=n_minor, noise=0.1, random_state=43, x_offset=0.3)

# データを結合
X = np.vstack((X_major, X_minor))
y = np.hstack((np.zeros(n_major), np.ones(n_minor)))

# データをシャッフル
shuffle_idx = np.random.permutation(len(X))
X, y = X[shuffle_idx], y[shuffle_idx]

# ラベルをint型に変換
y = y.astype(int)

# アンダーサンプリング手法の設定
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
lgbm_classifier = LGBMClassifier(n_estimators=100, reg_alpha=0.1, random_state=42)

iht_rf = InstanceHardnessThreshold(estimator=rf_classifier, sampling_strategy=target_ratio, random_state=42)
iht_lgbm = InstanceHardnessThreshold(estimator=lgbm_classifier, sampling_strategy=target_ratio, random_state=42)
nm1 = NearMiss(version=1, sampling_strategy=target_ratio)
nm2 = NearMiss(version=2, sampling_strategy=target_ratio)
nm3 = NearMiss(version=3, sampling_strategy=target_ratio)
enn = EditedNearestNeighbours(sampling_strategy='majority')
tl = TomekLinks(sampling_strategy='majority')
allknn = AllKNN(sampling_strategy='majority')
cc = ClusterCentroids(sampling_strategy=target_ratio, random_state=42)

# アンダーサンプリングの実行
X_iht_rf, y_iht_rf = iht_rf.fit_resample(X, y)
X_iht_lgbm, y_iht_lgbm = iht_lgbm.fit_resample(X, y)
X_nm1, y_nm1 = nm1.fit_resample(X, y)
X_nm2, y_nm2 = nm2.fit_resample(X, y)
X_nm3, y_nm3 = nm3.fit_resample(X, y)
X_enn, y_enn = enn.fit_resample(X, y)
X_tl, y_tl = tl.fit_resample(X, y)
X_allknn, y_allknn = allknn.fit_resample(X, y)
X_cc, y_cc = cc.fit_resample(X, y)

# プロット作成（2行5列）
fig, axes = plt.subplots(2, 5, figsize=(25, 10))

x_min, x_max = X[:, 0].min() - 0.5, X[:, 0].max() + 0.5
y_min, y_max = X[:, 1].min() - 0.5, X[:, 1].max() + 0.5

def plot_data(ax, X, y, title):
    ax.scatter(X[y==0, 0], X[y==0, 1], label='Normal (Majority)', alpha=0.7)
    ax.scatter(X[y==1, 0], X[y==1, 1], label='Fault (Minority)', alpha=0.7)
    ax.set_title(title)
    ax.legend()
    ax.set_xlim(x_min, x_max)
    ax.set_ylim(y_min, y_max)

plot_data(axes[0, 0], X, y, 'Original Data')
plot_data(axes[0, 1], X_iht_rf, y_iht_rf, 'After IHT (RF)')
plot_data(axes[0, 2], X_iht_lgbm, y_iht_lgbm, 'After IHT (LGBM)')
plot_data(axes[0, 3], X_nm1, y_nm1, 'After Nearmiss1')
plot_data(axes[0, 4], X_nm2, y_nm2, 'After Nearmiss2')
plot_data(axes[1, 0], X_nm3, y_nm3, 'After Nearmiss3')
plot_data(axes[1, 1], X_enn, y_enn, 'After ENN')
plot_data(axes[1, 2], X_tl, y_tl, 'After Tomek Links')
plot_data(axes[1, 3], X_allknn, y_allknn, 'After AllKNN')
plot_data(axes[1, 4], X_cc, y_cc, 'After ClusterCentroids')

plt.tight_layout()
plt.show()

# 比率を計算する関数
def calculate_ratio(y):
    return sum(y==1) / sum(y==0)

# クラスごとのサンプル数と比率を表示
print(f"目標比率: {target_ratio:.2f}")
print(f"Original data - Normal: {sum(y==0)}, Fault: {sum(y==1)}, 比率: {calculate_ratio(y):.2f}")
print(f"After IHT (RF) - Normal: {sum(y_iht_rf==0)}, Fault: {sum(y_iht_rf==1)}, 比率: {calculate_ratio(y_iht_rf):.2f}")
print(f"After IHT (LGBM) - Normal: {sum(y_iht_lgbm==0)}, Fault: {sum(y_iht_lgbm==1)}, 比率: {calculate_ratio(y_iht_lgbm):.2f}")
print(f"After Nearmiss1 - Normal: {sum(y_nm1==0)}, Fault: {sum(y_nm1==1)}, 比率: {calculate_ratio(y_nm1):.2f}")
print(f"After Nearmiss2 - Normal: {sum(y_nm2==0)}, Fault: {sum(y_nm2==1)}, 比率: {calculate_ratio(y_nm2):.2f}")
print(f"After Nearmiss3 - Normal: {sum(y_nm3==0)}, Fault: {sum(y_nm3==1)}, 比率: {calculate_ratio(y_nm3):.2f}")
print(f"After ENN - Normal: {sum(y_enn==0)}, Fault: {sum(y_enn==1)}, 比率: {calculate_ratio(y_enn):.2f}")
print(f"After Tomek Links - Normal: {sum(y_tl==0)}, Fault: {sum(y_tl==1)}, 比率: {calculate_ratio(y_tl):.2f}")
print(f"After AllKNN - Normal: {sum(y_allknn==0)}, Fault: {sum(y_allknn==1)}, 比率: {calculate_ratio(y_allknn):.2f}")
print(f"After ClusterCentroids - Normal: {sum(y_cc==0)}, Fault: {sum(y_cc==1)}, 比率: {calculate_ratio(y_cc):.2f}")
